In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rlway.pyosrd import OSRD
OSRD.use_cases

['c1',
 'c1_with_blocks',
 'c1y2',
 'c2x2',
 'c2xx2',
 'c2y13s',
 'c2y1sy2sy1s',
 'c2y1y2',
 'c2z2z2',
 'c3y1',
 'c3yy1yy3',
 'cvg_dvg',
 'point_switch',
 'station_capacity2',
 'straight_line']

In [3]:
use_case='c2y1y2y'
sim = OSRD(use_case=use_case, dir='tmp')

ValueError: c2y1y2y is not a valid use case name.

In [ ]:
sim.draw_infra_points()

In [ ]:
sim.space_time_chart_plotly(train=0)

In [ ]:
from rlway.schedules import schedule_from_osrd

In [ ]:
s = schedule_from_osrd(sim, eco_or_base='base')
ax = s.plot()
# ax.invert_yaxis()
# ax.legend('')
s.df

In [ ]:
s.draw_graph()

In [ ]:
# sim = use_case_station_capacity2
sim.reset_delays()
sim.add_delay('train0', time_threshold=10, delay=120.)
delayed = sim.delayed()

s = schedule_from_osrd(delayed, eco_or_base='base')
ax = s.plot()

In [ ]:
from rlway.pyosrd.scheduler_agent import SchedulerAgent
import pandas as pd
from dataclasses import dataclass

In [ ]:

class DummySchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 60s more at station"""
        extra_delays = self.initial_schedule.durations * 0.
        extra_delays.iloc[3][1] = 60.
        return extra_delays

@dataclass
class AddStop(SchedulerAgent):
    train: int = 0
    zone: str = ""
    duration: float = 0.

    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """Make the train stop at the given position for the given duration"""
        extra_delays = self.initial_schedule.durations * 0.
        extra_delays.loc[self.zone][self.train] = self.duration

        return extra_delays


In [ ]:
agent = AddStop('addstop', train=0, zone='D0<->buffer_stop.0', duration=50.)
agent.set_schedules_from_osrd(sim)

agent.initial_schedule.plot()
agent.delayed_schedule.plot()
agent.regulated_schedule.plot()


In [ ]:
from rlway_cpagent.ortools_agent.ortools_agent import OrtoolsAgent

In [ ]:
regulated = delayed.regulate(agent=OrtoolsAgent("cp_agent"))

In [ ]:
regulated_schedule = schedule_from_osrd(regulated)